#Simple audio recognition: Recognizing keywords

#Setup
Nhập các mô-đun và phụ thuộc cần thiết. Bạn sẽ sử dụng tf.keras.utils.audio_dataset_from_directory (được giới thiệu trong TensorFlow 2.10), giúp tạo tập dữ liệu phân loại âm thanh từ các thư mục của tệp .wav. Bạn cũng sẽ cần seaborn để hình dung trong hướng dẫn này.

In [ ]:
!pip install -U -q tensorflow tensorflow_datasets

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import the mini Speech Commands dataset


In [ ]:
DATASET_PATH = '/content/drive/MyDrive/VOICE/newdata'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

In [ ]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print('Commands:', commands)

Được chia thành các thư mục theo cách này, bạn có thể dễ dàng tải dữ liệu bằng keras.utils.audio_dataset_from_directory.

Các clip âm thanh có thời lượng từ 1 giây trở xuống ở tần số 16kHz. Output_sequence_length=16000 đệm những cái ngắn đến chính xác 1 giây (và sẽ cắt bớt những cái dài hơn) để chúng có thể dễ dàng được phân nhóm.

In [ ]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)

Tập dữ liệu hiện chứa nhiều clip âm thanh và nhãn số nguyên. Các clip âm thanh có hình dạng  (batch, samples, channels).

In [ ]:
train_ds.element_spec

Tập dữ liệu này chỉ chứa âm thanh một kênh, vì vậy hãy sử dụng hàm tf.squeeze để loại bỏ trục phụ:

In [ ]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

Hàm utils.audio_dataset_from_directory chỉ trả về tối đa hai phần tách. Bạn nên tách biệt bộ kiểm tra với bộ xác thực của mình. Lý tưởng nhất là bạn nên giữ nó trong một thư mục riêng, nhưng trong trường hợp này, bạn có thể sử dụng Dataset.shard để chia bộ xác thực thành hai nửa. Lưu ý rằng việc lặp lại bất kỳ phân đoạn nào sẽ tải tất cả dữ liệu và chỉ giữ lại phần của nó.

In [ ]:
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)

In [ ]:
for example_audio, example_labels in train_ds.take(1):
  print(example_audio.shape)
  print(example_labels.shape)

Hãy vẽ một vài dạng sóng âm thanh:

In [ ]:
plt.figure(figsize=(16, 10))
rows = 3
cols = 3
n = rows * cols
for i in range(n):
  plt.subplot(rows, cols, i+1)
  audio_signal = example_audio[i]
  plt.plot(audio_signal)
  plt.title(label_names[example_labels[i]])
  plt.yticks(np.arange(-1.2, 1.2, 0.2))
  plt.ylim([-1.1, 1.1])

#Convert waveforms to spectrograms
Các dạng sóng trong tập dữ liệu được biểu diễn trong miền thời gian. Tiếp theo, bạn sẽ chuyển đổi dạng sóng từ tín hiệu miền thời gian thành tín hiệu miền tần số thời gian bằng cách tính toán biến đổi Fourier thời gian ngắn (STFT) để chuyển đổi dạng sóng thành biểu đồ phổ, hiển thị sự thay đổi tần số theo thời gian và có thể được được biểu diễn dưới dạng hình ảnh 2D. Bạn sẽ đưa các hình ảnh phổ vào mạng lưới thần kinh của mình để huấn luyện mô hình.

Biến đổi Fourier (tf.signal.fft) chuyển đổi tín hiệu thành các tần số thành phần của nó nhưng mất toàn bộ thông tin về thời gian. Để so sánh, STFT (tf.signal.stft) chia tín hiệu thành các cửa sổ thời gian và chạy biến đổi Fourier trên mỗi cửa sổ, lưu giữ một số thông tin về thời gian và trả về một tenxơ 2D mà bạn có thể chạy các tích chập tiêu chuẩn trên đó.

Tạo hàm tiện ích để chuyển đổi dạng sóng thành biểu đồ phổ:

Các dạng sóng cần phải có cùng độ dài để khi bạn chuyển đổi chúng thành biểu đồ phổ, kết quả có kích thước tương tự nhau. Điều này có thể được thực hiện bằng cách đơn giản thêm khoảng đệm vào các clip âm thanh ngắn hơn một giây (sử dụng tf.zeros).
Khi gọi tf.signal.stft, hãy chọn tham số frame_length và frame_step sao cho "hình ảnh" phổ được tạo gần như hình vuông. Để biết thêm thông tin về lựa chọn tham số STFT, hãy tham khảo video Coursera này về xử lý tín hiệu âm thanh và STFT.
STFT tạo ra một dãy số phức biểu thị cường độ và pha. Tuy nhiên, trong hướng dẫn này, bạn sẽ chỉ sử dụng độ lớn mà bạn có thể rút ra bằng cách áp dụng tf.abs trên đầu ra của tf.signal.stft.

In [ ]:
def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

Next, start exploring the data. Print the shapes of one example's tensorized waveform and the corresponding spectrogram, and play the original audio

In [ ]:
for i in range(3):
  label = label_names[example_labels[i]]
  waveform = example_audio[i]
  spectrogram = get_spectrogram(waveform)

  print('Label:', label)
  print('Waveform shape:', waveform.shape)
  print('Spectrogram shape:', spectrogram.shape)
  print('Audio playback')
  display.display(display.Audio(waveform, rate=16000))

Bây giờ, hãy xác định hàm để hiển thị biểu đồ phổ:

In [ ]:
def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

Plot the example's waveform over time and the corresponding spectrogram (frequencies over time):

In [ ]:
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(waveform.shape[0])
axes[0].plot(timescale, waveform.numpy())
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 16000])

plot_spectrogram(spectrogram.numpy(), axes[1])
axes[1].set_title('Spectrogram')
plt.suptitle(label.title())
plt.show()

Bây giờ, hãy tạo bộ dữ liệu biểu đồ phổ từ bộ dữ liệu âm thanh:

In [ ]:
def make_spec_ds(ds):
  return ds.map(
      map_func=lambda audio,label: (get_spectrogram(audio), label),
      num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train_spectrogram_ds = make_spec_ds(train_ds)
val_spectrogram_ds = make_spec_ds(val_ds)
test_spectrogram_ds = make_spec_ds(test_ds)

Kiểm tra các biểu đồ phổ cho các ví dụ khác nhau của tập dữ liệu:

In [ ]:
for example_spectrograms, example_spect_labels in train_spectrogram_ds.take(1):
  break

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(16, 9))

for i in range(n):
    r = i // cols
    c = i % cols
    ax = axes[r][c]
    plot_spectrogram(example_spectrograms[i].numpy(), ax)
    ax.set_title(label_names[example_spect_labels[i].numpy()])

plt.show()

#Build and train the model

Thêm các thao tác `Dataset.cache` và `Dataset.prefetch` để giảm độ trễ đọc trong khi đào tạo mô hình:

In [ ]:
train_spectrogram_ds = train_spectrogram_ds.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = val_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = test_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)

Đối với mô hình này, bạn sẽ sử dụng mạng thần kinh tích chập (CNN) đơn giản vì bạn đã chuyển đổi các tệp âm thanh thành hình ảnh phổ.

Mô hình `tf.keras.Sequential` của bạn sẽ sử dụng các lớp tiền xử lý Keras sau:

- `tf.keras.layers.Resizing`: giảm mẫu đầu vào để cho phép mô hình đào tạo nhanh hơn.
- `tf.keras.layers.Normalization`: để chuẩn hóa từng pixel trong ảnh dựa trên giá trị trung bình và độ lệch chuẩn của nó.

Đối với lớp `Chuẩn hóa`, phương pháp `thích ứng` của nó trước tiên cần được gọi trên dữ liệu huấn luyện để tính toán số liệu thống kê tổng hợp (nghĩa là giá trị trung bình và độ lệch chuẩn).

In [ ]:
input_shape = example_spectrograms.shape[1:]
print('Input shape:', input_shape)
num_labels = len(label_names)

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
# Fit the state of the layer to the spectrograms
# with `Normalization.adapt`.
norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

model = models.Sequential([
    layers.Input(shape=input_shape),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

Định cấu hình mô hình Keras với trình tối ưu hóa Adam và mất entropy chéo:

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

Train the model over 10 epochs for demonstration purposes:

In [ ]:
EPOCHS = 10
history = model.fit(
    train_spectrogram_ds,
    validation_data=val_spectrogram_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

In [ ]:
metrics = history.history
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch')
plt.ylabel('Loss [CrossEntropy]')

plt.subplot(1,2,2)
plt.plot(history.epoch, 100*np.array(metrics['accuracy']), 100*np.array(metrics['val_accuracy']))
plt.legend(['accuracy', 'val_accuracy'])
plt.ylim([0, 100])
plt.xlabel('Epoch')
plt.ylabel('Accuracy [%]')

#Đánh giá hiệu quả mô hình
Run the model on the test set and check the model's performance:

In [ ]:
model.evaluate(test_spectrogram_ds, return_dict=True)

#Display a confusion matrix

In [ ]:
y_pred = model.predict(test_spectrogram_ds)

In [ ]:
y_pred = tf.argmax(y_pred, axis=1)

In [ ]:
y_true = tf.concat(list(test_spectrogram_ds.map(lambda s,lab: lab)), axis=0)

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=label_names,
            yticklabels=label_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

# Chạy suy luận trên tệp âm thanh

In [ ]:
x = data_dir/'/content/drive/MyDrive/VOICE/newdata/Nam/audio37_converted.wav'
x = tf.io.read_file(str(x))
x, sample_rate = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000,)
x = tf.squeeze(x, axis=-1)
waveform = x
x = get_spectrogram(x)
x = x[tf.newaxis,...]

prediction = model(x)
x_labels = ['Bac','Trung','Nam']
plt.bar(x_labels, tf.nn.softmax(prediction[0]))
plt.title('Bac')
plt.show()

display.display(display.Audio(waveform, rate=16000))

#Export the model with preprocessing

Mô hình này không dễ sử dụng nếu bạn phải áp dụng các bước tiền xử lý đó trước khi chuyển dữ liệu sang mô hình để suy luận. Vì vậy, hãy xây dựng một phiên bản end-to-end:

In [ ]:
class ExportModel(tf.Module):
  def __init__(self, model):
    self.model = model

    # Accept either a string-filename or a batch of waveforms.
    # YOu could add additional signatures for a single wave, or a ragged-batch.
    self.__call__.get_concrete_function(
        x=tf.TensorSpec(shape=(), dtype=tf.string))
    self.__call__.get_concrete_function(
       x=tf.TensorSpec(shape=[None, 16000], dtype=tf.float32))


  @tf.function
  def __call__(self, x):
    # If they pass a string, load the file and decode it.
    if x.dtype == tf.string:
      x = tf.io.read_file(x)
      x, _ = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000,)
      x = tf.squeeze(x, axis=-1)
      x = x[tf.newaxis, :]

    x = get_spectrogram(x)
    result = self.model(x, training=False)

    class_ids = tf.argmax(result, axis=-1)
    class_names = tf.gather(label_names, class_ids)
    return {'predictions':result,
            'class_ids': class_ids,
            'class_names': class_names}

Test run the "export" model:

In [ ]:
export = ExportModel(model)
export(tf.constant(str(data_dir/'/content/drive/MyDrive/VOICE/newdata/Nam/audio37_converted.wav')))

In [ ]:
tf.saved_model.save(export, "saved")
imported = tf.saved_model.load("saved")
imported(waveform[tf.newaxis, :])